In [15]:
!pip install kaggle

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/82.1 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/82.1 kB ? eta -:--:--
     -------------- ----------------------- 30.7/82.1 kB 325.1 kB/s eta 0:00:01
     -------------- ----------------------- 30.7/82.1 kB 325.1 kB/s eta 0:00:01
     ---------------------------- --------- 61.4/82.1 kB 326.1 kB/s eta 0:00:01
     -------------------------------------- 82.1/82.1 kB 352.2 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kaggle: filename=kaggle-1.6.14-py3-none-any.whl size=105134 sha256=2e69759dc2e246da84ce8da4c042be00181ca43afba542c4ad52dab0ea2e0868
  Stored in directory: c:\users\alberreal\appdata\local\pip\cache\wheels\ec\d9\04\93be57edf3835182a8e76d8ff87f7e77a0113f473ba3721ee0
Successfully built kaggle


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

In [2]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to the desired size (e.g., 224x224 for ResNet)
    transforms.ToTensor(),          # Convert image to PyTorch tensor
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet values
    transforms.Normalize(mean=[0.4581, 0.4581, 0.4581], std=[0.2589, 0.2589, 0.2589])  # Values for this dataset
])



In [3]:
# Load the dataset
dataset = ImageFolder(root='mini child dataset', transform=transform)
transform = transforms.Compose([
    transforms.ToTensor()
])

In [4]:
dataset

Dataset ImageFolder
    Number of datapoints: 10
    Root location: mini child dataset
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.4581, 0.4581, 0.4581], std=[0.2589, 0.2589, 0.2589])
           )

In [ ]:
#finding mean and std for images
transform = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
dataset = ImageFolder(root='mini child dataset', transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


In [ ]:
mean = torch.zeros(3)
std = torch.zeros(3)
n_samples = 0

In [ ]:
for images, _ in dataloader:
    # Resize images to [N, 3, H, W]
    images = images.view(images.size(0), images.size(1), -1)
    # Update the sum and sum of squares
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    n_samples += images.size(0)

In [ ]:
mean /= n_samples
std /= n_samples

In [ ]:
print("Mean:", mean)
print("Std:", std)

In [ ]:
#end of finding mean and std

In [5]:
# Split the dataset into training and test sets (e.g., 80% training, 20% testing)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
#train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_ds, valid_ds = random_split(dataset, [train_size, test_size])

In [ ]:
len(list(train_ds)[0][0][0][0])

In [6]:
# Create data loaders
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(valid_ds, batch_size=64, shuffle=False)


In [7]:
# Verify the dataset structure and labels
print(f'Total images: {len(dataset)}')
print(f'Training images: {len(train_ds)}')
print(f'Testing images: {len(valid_ds)}')

Total images: 10
Training images: 8
Testing images: 2


In [8]:
for images, labels in train_loader:
    print(images.shape)  # Batch size x Channels x Height x Width
    print(labels)        # Labels for the batch
    break  # Just checking one batch

torch.Size([8, 3, 224, 224])
tensor([1, 1, 0, 0, 1, 0, 1, 0])


In [9]:
import torch

from dataset import *
from model import ResNet18
from unlearn import *
from metrics import UnLearningScore
from utils import *
from torch.utils.data import DataLoader

C:\ProgramData\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [10]:
#train_ds = CustomCIFAR100(root='.', train=True,download=True, transform=transform_train)
#valid_ds = CustomCIFAR100(root='.', train=False,download=True, transform=transform_train)

batch_size = 256
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=32, pin_memory=True)
valid_dl = DataLoader(valid_ds, batch_size, num_workers=32, pin_memory=True)

C:\ProgramData\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:558: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
train_dataset[0]

In [11]:
"""num_classes = 100
classwise_train = {}
for i in range(num_classes):
    classwise_train[i] = []

for img, label, clabel in train_ds:
    classwise_train[label].append((img, label, clabel))
    
classwise_test = {}
for i in range(num_classes):
    classwise_test[i] = []

for img, label, clabel in valid_ds:
    classwise_test[label].append((img, label, clabel))"""
    
num_classes = 2
classwise_train = {}
for i in range(num_classes):
    classwise_train[i] = []

for img, label in train_ds:
    classwise_train[label].append((img, label))
    
classwise_test = {}
for i in range(num_classes):
    classwise_test[i] = []

for img, label in valid_ds:
    classwise_test[label].append((img, label))

In [12]:
# train the model
#device = 'cuda'
device = 'cpu'
                                                                                                                                                                                                                               
epochs = 5
history = fit_one_cycle(epochs, model, train_dl, valid_dl, device = device)
torch.save(model.state_dict(), "ResNET18_ChildNonChild_Pretrained_ALL_CLASSES_5_Epochs.pt")

C:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\ProgramData\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [0], last_lr: 0.00100, train_loss: 0.7966, val_loss: 0.8231, val_acc: 50.0000
Epoch [1], last_lr: 0.00100, train_loss: 0.0268, val_loss: 1.2741, val_acc: 50.0000
Epoch [2], last_lr: 0.00100, train_loss: 0.0190, val_loss: 2.1944, val_acc: 50.0000
Epoch [3], last_lr: 0.00100, train_loss: 0.0011, val_loss: 3.0285, val_acc: 50.0000
Epoch [4], last_lr: 0.00100, train_loss: 0.0002, val_loss: 3.9321, val_acc: 50.0000


In [ ]:
print ("hi")

In [13]:
# load the trained model
#device = 'cuda'
model = ResNet18(num_classes = 2, pretrained = True).to(device)
model.load_state_dict(torch.load("ResNET18_ChildNonChild_Pretrained_ALL_CLASSES_5_Epochs.pt", map_location='cpu'))

<All keys matched successfully>

# Forgetting Rocket
The Rocket is class 69 in CIFAR100 and belongs to Super Class 19 (Vehicles) in CIFAR Super 20.

In [14]:
# Getting the forget and retain validation data
"""forget_valid = []
forget_classes = [0]
for cls in range(num_classes):
    if cls in forget_classes:
        for img, label, clabel in classwise_test[cls]:
            forget_valid.append((img, label, clabel))

retain_valid = []
for cls in range(num_classes):
    if cls not in forget_classes:
        for img, label, clabel in classwise_test[cls]:
            retain_valid.append((img, label, clabel))
            
forget_train = []
for cls in range(num_classes):
    if cls in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            forget_train.append((img, label, clabel))

retain_train = []
for cls in range(num_classes):
    if cls not in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            retain_train.append((img, label, clabel))"""
forget_valid = []
forget_classes = [0]
for cls in range(num_classes):
    if cls in forget_classes:
        for img, label in classwise_test[cls]:
            forget_valid.append((img, label))

retain_valid = []
for cls in range(num_classes):
    if cls not in forget_classes:
        for img, label in classwise_test[cls]:
            retain_valid.append((img, label))
            
forget_train = []
for cls in range(num_classes):
    if cls in forget_classes:
        for img, label in classwise_train[cls]:
            forget_train.append((img, label))

retain_train = []
for cls in range(num_classes):
    if cls not in forget_classes:
        for img, label in classwise_train[cls]:
            retain_train.append((img, label))

forget_valid_dl = DataLoader(forget_valid, batch_size, num_workers=32, pin_memory=True)

retain_valid_dl = DataLoader(retain_valid, batch_size, num_workers=32, pin_memory=True)

forget_train_dl = DataLoader(forget_train, batch_size, num_workers=32, pin_memory=True)
retain_train_dl = DataLoader(retain_train, batch_size, num_workers=32, pin_memory=True, shuffle = True)
import random
retain_train_subset = random.sample(retain_train, int(0.3*len(retain_train)))
retain_train_subset_dl = DataLoader(retain_train_subset, batch_size, num_workers=32, pin_memory=True, shuffle = True)

In [ ]:
# Performance of Fully trained model on retain set
evaluate(model, retain_valid_dl, device)

In [ ]:
# Performance of Fully trained model on retain set
evaluate(model, forget_valid_dl, device)

## Retrain the model from Scratch
Create Retrained Model (Gold model). This is the model trained from scratch without forget data.

In [ ]:
device = 'cuda'
gold_model = ResNet18(num_classes = 20, pretrained = True).to(device)
epochs = 5
history = fit_one_cycle(epochs, gold_model, retain_train_dl, retain_valid_dl, device = device)
torch.save(gold_model.state_dict(), "ResNET18_CIFAR100Super20_Pretrained_Gold_Class69_5_Epochs.pt")

In [ ]:
device = 'cuda'
gold_model = ResNet18(num_classes = 20, pretrained = True).to(device)
gold_model.load_state_dict(torch.load("ResNET18_CIFAR100Super20_Pretrained_Gold_Class69_5_Epochs.pt", map_location=device))

In [ ]:
# evaluate gold model on forget set
evaluate(gold_model, forget_valid_dl, device)

In [ ]:
# evaluate gold model on retain set
evaluate(gold_model, retain_valid_dl, device)

## UnLearning via proposed method

In [ ]:
device = 'cuda'
unlearning_teacher = ResNet18(num_classes = 20, pretrained = False).to(device).eval()
student_model = ResNet18(num_classes = 20, pretrained = False).to(device)
student_model.load_state_dict(torch.load("ResNET18_ChildNonChild_Pretrained_ALL_CLASSES_5_Epochs.pt", map_location = device))
model = model.eval()

KL_temperature = 1

optimizer = torch.optim.Adam(student_model.parameters(), lr = 0.0001)

blindspot_unlearner(model = student_model, unlearning_teacher = unlearning_teacher, full_trained_teacher = model, 
          retain_data = retain_train_subset, forget_data = forget_train, epochs = 1, optimizer = optimizer, lr = 0.0001, 
          batch_size = 256, num_workers = 32, device = device, KL_temperature = KL_temperature)

In [ ]:
# performance of unlearned model on forget set
evaluate(student_model, forget_valid_dl, device)

In [ ]:
# performance of unlearned model on retain set
evaluate(student_model, retain_valid_dl, device)

### Measure ZRF (Unlearning Score)

In [ ]:
print("Initial Score: {}".format(UnLearningScore(model, unlearning_teacher, forget_valid_dl, 256, 'cuda')))
print("Our Score: {}".format(UnLearningScore(student_model, unlearning_teacher, forget_valid_dl, 256, 'cuda')))
print("Gold Score: {}".format(UnLearningScore(gold_model, unlearning_teacher, forget_valid_dl, 256, 'cuda')))
print("JS Div: {}".format(1-UnLearningScore(gold_model, student_model, forget_valid_dl, 256, 'cuda')))

## Unlearning using Amnesiac unlearning

In [ ]:
unlearninglabels = list(range(20))
unlearninglabels.remove(19)
unlearning_train_set = []
for cls in range(num_classes):
    if cls in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            unlearning_train_set.append((img, label, random.choice(unlearninglabels)))



for cls in range(num_classes):
    if cls not in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            unlearning_train_set.append((img, label, clabel))

In [ ]:
unlearning_train_set_dl = DataLoader(unlearning_train_set, batch_size, num_workers = 32, pin_memory = True, shuffle = True)

In [ ]:
device = 'cuda'
student_model = ResNet18(num_classes = 20, pretrained = True).to(device)
student_model.load_state_dict(torch.load("ResNET18_ChildNonChild_Pretrained_ALL_CLASSES_5_Epochs.pt", map_location = 'cuda'))
epochs = 3

history = fit_one_unlearning_cycle(epochs, student_model, unlearning_train_set_dl, retain_valid_dl, device = device, lr = 0.0001)

In [ ]:
print("Forget Performance: {}".format(evaluate(student_model, forget_valid_dl, device)))
print("Retain Performance: {}".format(evaluate(student_model, retain_valid_dl, device)))

## Unlearning using UNSIR (Class 0)

In [ ]:
num_classes = 20
classwise_train = {}
for i in range(num_classes):
    classwise_train[i] = []

for img, label, clabel in train_ds:
    classwise_train[clabel].append((img, label, clabel))
    
classwise_test = {}
for i in range(num_classes):
    classwise_test[i] = []

for img, label, clabel in valid_ds:
    classwise_test[clabel].append((img, label, clabel))

In [ ]:
# Getting the forget and retain validation data
forget_valid = []
forget_classes = [0]
for cls in range(num_classes):
    if cls in forget_classes:
        for img, label, clabel in classwise_test[cls]:
            forget_valid.append((img, label, clabel))

retain_valid = []
for cls in range(num_classes):
    if cls not in forget_classes:
        for img, label, clabel in classwise_test[cls]:
            retain_valid.append((img, label, clabel))
            
forget_train = []
for cls in range(num_classes):
    if cls in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            forget_train.append((img, label, clabel))

retain_train = []
for cls in range(num_classes):
    if cls not in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            retain_train.append((img, label, clabel))

forget_valid_dl = DataLoader(forget_valid, batch_size, num_workers=32, pin_memory=True)

retain_valid_dl = DataLoader(retain_valid, batch_size, num_workers=32, pin_memory=True)

forget_train_dl = DataLoader(forget_train, batch_size, num_workers=32, pin_memory=True)
retain_train_dl = DataLoader(retain_train, batch_size, num_workers=32, pin_memory=True, shuffle = True)

In [ ]:
#collect some samples from each class
num_samples = 500
retain_samples = []
for i in range(num_classes):
    if i not in forget_classes:
        retain_samples += classwise_train[i][:num_samples]

In [ ]:
noise_batch_size = 256

In [ ]:
device = 'cuda'
student_model = ResNet18(num_classes = 20, pretrained = False).to(device)
student_model.load_state_dict(torch.load("ResNET18_ChildNonChild_Pretrained_ALL_CLASSES_5_Epochs.pt", map_location = device))

In [ ]:
noise = UNSIR_noise(batch_size, 3, 224, 224).to(device)

In [ ]:
forget_class_label = 0
num_epochs = 250

In [ ]:
noise =  UNSIR_noise_train(noise, student_model, forget_class_label, num_epochs,\
                           noise_batch_size, device=device)

In [ ]:
noisy_loader = UNSIR_create_noisy_loader(noise, forget_class_label\
                                         , retain_samples, batch_size, device=device)

In [ ]:
#impair step
epochs = 1
history = fit_one_unlearning_cycle(epochs, student_model, noisy_loader, retain_valid_dl, device = device, lr = 0.0001)

In [ ]:
print("Forget Performance: {}".format(evaluate(student_model, forget_valid_dl, device)))
print("Retain Performance: {}".format(evaluate(student_model, retain_valid_dl, device)))

In [ ]:
#repair step
other_samples = []
for i in range(len(retain_samples)):
    other_samples.append((retain_samples[i][0].cpu(), torch.tensor(retain_samples[i][2]),
                            torch.tensor(retain_samples[i][2])))    

heal_loader = torch.utils.data.DataLoader(other_samples, batch_size=batch_size, shuffle = True)
epochs = 1
history = fit_one_unlearning_cycle(epochs, student_model, heal_loader, retain_valid_dl, device = device, lr = 0.0001)

In [ ]:
print("Forget Performance: {}".format(evaluate(student_model, forget_valid_dl, device)))
print("Retain Performance: {}".format(evaluate(student_model, retain_valid_dl, device)))